In [1]:
pip install -U langchain-community langchain-mistralai ollama faiss-cpu

In [2]:
mistral_api_key = # PUT HERE YOUR API KEY

In [8]:
import os
from typing import Optional, List, Tuple
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings, OllamaEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama, VLLM, HuggingFaceHub, OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import glob
from langchain_mistralai import ChatMistralAI
import numpy as np
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [4]:
def load_documents(directory):
    """
    Загружаем текстовые документы из файлов
    """
    documents = []
    for file_path in glob.glob(os.path.join(directory, "lecture*.txt")):
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            documents.extend(loader.load())
            print(file_path, end=' ')
        except Exception as e:
            print(f"Не удалось загрузить файл {file_path}: {e}")
    return documents

data_directory = "./data"
documents = load_documents(data_directory)

./data/lecture1.txt ./data/lecture8.txt ./data/lecture2.txt ./data/lecture9.txt ./data/lecture10.txt ./data/lecture14.txt ./data/lecture11.txt ./data/lecture13.txt ./data/lecture5.txt ./data/lecture7.txt ./data/lecture6.txt ./data/lecture4.txt ./data/lecture12.txt ./data/lecture3.txt 

In [5]:
MARKDOWN_SEPARATORS = [
    "\subsection",
    "\begin{exframe}",
    "\end{exframe}"
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_text(documents, chunk_size=2000, chunk_overlap=200):
    """
    Разбиваем текст на чанки с заданными размером и перекрытием.
    Используем RecursiveCharacterTextSplitter, поскольку он разбивает
    текст, учитывая знаки препинания и структуру текста
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

chunks = split_text(documents)

In [9]:
def create_embeddings(chunks, embedding_type="huggingface", model_name="all-MiniLM-L6-v2"):
    """
    Создаём эмбеддинги для текстовых чанков
    """
    if embedding_type == "huggingface":
        embeddings = HuggingFaceEmbeddings(model_name=model_name)
    elif embedding_type == "ollama":
        embeddings = OllamaEmbeddings(model="llama3")
    elif embedding_type == "openai":
        embeddings = OpenAIEmbeddings()
    else:
        raise ValueError(f"Unknown embedding type: {embedding_type}")

    db = FAISS.from_documents(chunks, embeddings)
    return db

db = create_embeddings(chunks, embedding_type="huggingface", model_name="all-MiniLM-L6-v2")

<ipython-input-9-34b54ddd729d>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
def create_rag_chain(db, model_name="mistral-large-latest"):
    """
    Создаём RAG-цепочку для получения ответов на вопросы пользователя
    """
    llm = ChatMistralAI(mistral_api_key=mistral_api_key, model=model_name, timeout=50)

    template = """
    Тебе даны текст и вопрос (смотри ниже). Нужно ответить на вопрос используя только информацию из предоставленного
    текста. При этом ты должен создать ощущение, что ты понимаешь, о чём этот текст.
    Ответ должен сохранять смысл, который был в тексте, при этом можно перефразировать предложения.
    Ничего добавлять от себя нельзя.
    Если в полученном тексте нет ответа, скажи, что ответа нет, не пытайся создать ответ самостоятельно.

    Текст: {context}

    Вопрос: {question}
    Ответ:
    """
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type="stuff",
                                          retriever=db.as_retriever(search_kwargs={"k": 5}),
                                          chain_type_kwargs={"prompt": prompt})
    return qa_chain


rag_chain = create_rag_chain(db, model_name='mistral-large-2402')


#def get_response(query):
#  response = rag_chain.run(query)
#  return response

In [11]:
def get_response(query, num_responses=3):
    """
    Генерируем несколько ответов на запрос и выбираем лучший
    с помощью функции choose_best_response
    """
    responses = []
    for _ in range(num_responses):
        responses.append(rag_chain.run(query))
    return choose_best_response(responses)

def choose_best_response(responses: List[str]) -> str:
    """
        Выбираем лучший ответ из списка
        Лучший = с наивысшей оценкой (произведение длины ответа и уникальности токенов)
    """
    if not responses:
        return "Нет ответов"

    def score_response(response):
        tokens = response.split()
        unique_tokens = set(tokens)
        return len(tokens) * len(unique_tokens)

    scores = [score_response(response) for response in responses]
    best_index = np.argmax(scores)
    return responses[best_index]


In [12]:
user_query = "Очень кратко расскажи, какие главные понятия содержит этот документ"
response = get_response(user_query)
print(f"Question: {user_query}")
print(f"Answer: {response}")

<ipython-input-11-576e4a070440>:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  responses.append(rag_chain.run(query))


Question: Очень кратко расскажи, какие главные понятия содержит этот документ
Answer: Главные понятия, содержащиеся в этом документе, включают:

1. Оптимизацию на "простых" множествах, проекцию и метод Франк-Вульфа.
2. Централизованный спуск и проблему коммуникационного узкого места в распределенном обучении.
3. Методы борьбы за эффективные коммуникации.
4. Одновременные и поочередные обновления переменных.
5. Метод поочередного градиентного спуска-подъема (Alt-GDA) и его алгоритм.
6. Проблемы сходимости методов градиентного спуска-подъема в не сильно выпуклых-сильно вогнутых задачах.
7. Метод внутренней точки как альтернативу методу барьеров.

Эти понятия относятся к области оптимизации и машинного обучения.


In [13]:
user_query = "Что такое градиентный спуск?"
response = get_response(user_query)
print(f"Question: {user_query}")
print(f"Answer: {response}")

user_query = "Есть ли другие документы на эту же тему?"
response = get_response(user_query)
print(f"Question: {user_query}")
print(f"Answer: {response}")

Question: Что такое градиентный спуск?
Answer: Градиентный спуск - это метод оптимизации, используемый для минимизации функции. Суть метода заключается в последовательном вычислении градиента функции в текущей точке и переходе в новую точку, которая выбирается в направлении противоположном градиенту. Целью является нахождение такой точки, в которой функция принимает минимальное значение. В тексте также упоминается вопрос о сходимости этого метода и о допустимых диапазонах шагов.
Question: Есть ли другие документы на эту же тему?
Answer: Из предоставленного текста невозможно узнать, есть ли другие документы на эту же тему. Текст содержит информацию о лекции, посвященной седловой задаче, методу экстраградиента, прямо-двойственному методу, градиентному спуску для гладких сильно выпуклых задач, барьерным функциям и другим связанным темам, но не содержит ссылок на другие документы.


**Простой Telegram Bot**

In [14]:
pip install --upgrade python-telegram-bot telebot

In [15]:
MY_TOKEN = # PUT HERE YOUR TELEGRAM BOT TOKEN

import telebot
import random

bot = telebot.TeleBot(MY_TOKEN)

@bot.message_handler(commands = ['start'])
def start(message):
  bot.send_message(message.chat.id, "Привет! Я бот, готовый ответить на твои вопросы по курсу Методы оптимизации МФТИ. Задай свой вопрос")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
  random_messages = ['Ну и вопрос! Подожди, пожалуйста, я подумаю', 'Подожди, где-то я уже такое видел... Проверяю', 'Дай подумать, минуточку',
                     'Хмм...', 'Сейчас-сейчас...', 'Надо подумать', 'О, это я знаю. Наверно :)', 'Сейчас я тебя впечатлю, подожди минуточку',
                     'Дай мне минутку, я обрабатываю твой вопрос', 'Кручу шестеренки...', 'Загрузка...', 'Соображаю...', 'Копаюсь в книге знаний',
                     'Консультируюсь с гением внутри меня', 'Перевариваю информацию...', 'Щелкаю пальцами в раздумьях', 'Подключаюсь к главному компьютеру...',
                     'Погружаюсь в глубины знаний...', 'Поджигаю свои нейроны', 'Катаюсь на ментальных американских горках', 'Пишу код, который тебе понравится',
                     'Вычисляю ответ с помощью супербыстрого мозга', 'Болтаю с другими ботами, чтобы получить идеи', 'Беру паузу, чтобы сварить себе чашечку кофе',
                     'Провожу тестирование на горячо любимых хомячках', 'Выполняю сложные алгоритмы...', 'Пытаюсь понять, что ты все-таки имеешь в виду...']
  send_thinking = random.choice(random_messages)
  bot.send_message(message.chat.id, send_thinking)
  try:
      response = get_response(message.text)
  except:
      bot.send_message(message.chat.id, "Мы превысили лимит токенов в минуту, подожди немного и повтори вопрос")
  bot.reply_to(message, 'Вот ответ на твой вопрос: \n' + response)

bot.polling()